In [2]:
import torch

!pip install optuna -q
!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y -q
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html -q
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html -q
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html -q
!pip install git+https://github.com/pyg-team/pytorch_geometric.git -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 16.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
# data & geospatial
import numpy as np
import pandas as pd
import geopandas as gpd
import os

# graph handling
import networkx as nx

# scikit‐learn utilities
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr

# progress bars
from tqdm.auto import tqdm

# miscellaneous
import random
from collections import Counter

# plotting
import matplotlib.pyplot as plt

# PyTorch & PyTorch‐Geometric
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv

# Mounting drive
from google.colab import drive

# This will prompt you to authorize Colab to access your Drive.
drive.mount('/content/drive')


Mounted at /content/drive


In [9]:
# ────────────────────────────────────────────────────────────
# 1) Load GeoDataFrame & build topological graph
# ────────────────────────────────────────────────────────────
fp = '/content/drive/MyDrive/Universiteit Utrecht/Thesis/data/road_network_lufeature.geojson'
gdf = gpd.read_file(fp).to_crs(epsg=28992).reset_index(drop=True)

# 2.2) reset index so `idx` is our node ID
gdf = gdf.reset_index(drop=True)

# 2.3) compute centroids & coords array
gdf['centroid'] = gdf.geometry.centroid
coords = np.vstack([ (pt.x, pt.y) for pt in gdf.centroid ])

# 2.4) choose your full list of features + target
feature_cols = [
    'AGRI_100','INDUS_100','NATUR_100','PORT_100','RES_100','TRANS_100','URBG_100','WATER_100',
    'AGRI_300','INDUS_300','NATUR_300','PORT_300','RES_300','TRANS_300','URBG_300','WATER_300',
    'AGRI_500','INDUS_500','NATUR_500','PORT_500','RES_500','TRANS_500','URBG_500','WATER_500',
    'AGRI_1000','INDUS_1000','NATUR_1000','PORT_1000','RES_1000','TRANS_1000','URBG_1000','WATER_1000',
    'AGRI_5000','AIR_5000','INDUS_5000','NATUR_5000','PORT_5000','RES_5000','TRANS_5000','URBG_5000','WATER_5000',
    'POP_100','POP_300','POP_500','POP_1000','POP_5000',
    'EEA_100','EEA_300','EEA_500','EEA_1000','EEA_5000',
    'HHOLD_100','HHOLD_300','HHOLD_500','HHOLD_1000',
    'RDL_25','TLOA_25','HLOA_25','MRDL_25','TMLOA_25','HMLOA_25',
    'RDL_50','TLOA_50','HLOA_50','MRDL_50','TMLOA_50','HMLOA_50',
    'RDL_100','TLOA_100','HLOA_100','MRDL_100','TMLOA_100','HMLOA_100',
    'RDL_300','TLOA_300','HLOA_300','MRDL_300','TMLOA_300','HMLOA_300',
    'RDL_500','TLOA_500','HLOA_500','MRDL_500','TMLOA_500','HMLOA_500',
    'RDL_1000','TLOA_1000','HLOA_1000','MRDL_1000','TMLOA_1000','HMLOA_1000',
    'TRAFNEAR','HTRAFNEAR','DINVNEAR','TRAFMAJOR','HTRAFMAJOR','DINVMAJOR'
]
target_col = 'NO2d'

# 2.5) Define your groups
groups = {
    'industrial':        ['INDUS_300','INDUS_1000'],
    'residential':       ['RES_300','RES_1000'],
    'agriculture':       ['AGRI_300','AGRI_1000'],
    'natural':           ['NATUR_300','NATUR_1000'],
    'port':              ['PORT_300','PORT_1000'],
    'urb_built':         ['URBG_300','URBG_1000'],
    'water':             ['WATER_300','WATER_1000'],
    'traffic':           ['TRAFNEAR','TRAFMAJOR'],
    'pop':               ['POP_300','POP_1000'],
    'population_density':['EEA_300','EEA_1000'],
}

def build_road_graph(gdf):
    G = nx.Graph()
    sidx = gdf.sindex
    for idx, row in gdf.iterrows():
        G.add_node(idx, **row.drop('geometry').to_dict())
    for idx, geom in enumerate(gdf.geometry):
        for j in sidx.intersection(geom.bounds):
            if idx != j and geom.touches(gdf.geometry[j]):
                G.add_edge(idx, j)
    return G

G = build_road_graph(gdf)

# ────────────────────────────────────────────────────────────
# 2) Precompute feature matrix & coordinates (must match gdf)
# ────────────────────────────────────────────────────────────
feature_matrix = gdf[feature_cols].to_numpy()  # shape (N, F)
coords = np.column_stack((
    gdf.geometry.centroid.x.values,
    gdf.geometry.centroid.y.values
))

# ────────────────────────────────────────────────────────────
# 3) Augmentation function (robust to missing cols)
# ────────────────────────────────────────────────────────────
from collections import Counter
from sklearn.neighbors import NearestNeighbors

def augment_grouped_far_knn(
    G, gdf, groups, coords, feature_matrix, feature_cols,
    top_n=500, neighbors=50, sim_thresh=0.9,
    min_dist=200.0, max_dist=2000.0,
    hop_thresh=3, max_edges=3000, per_node_cap=5,
    road_id_col="ROAD_FID", suffix='grp_far_knn'
):
    road_ids = gdf[road_id_col].to_numpy()
    col_to_idx = {c:i for i,c in enumerate(feature_cols)}
    candidates = set()

    for grp, cols in groups.items():
        # skip if any group-col not in feature_cols
        if any(c not in col_to_idx for c in cols):
            continue

        intensity = gdf[cols].sum(axis=1)
        top_idx   = intensity.nlargest(top_n).index.to_numpy()
        if top_idx.size < 2:
            continue

        idxs = [col_to_idx[c] for c in cols]
        subF = feature_matrix[top_idx][:, idxs]
        norms = np.linalg.norm(subF, axis=1, keepdims=True).clip(min=1e-6)
        subF = subF / norms

        nbr = NearestNeighbors(
            n_neighbors=min(neighbors+1, len(top_idx)),
            metric='cosine'
        ).fit(subF)
        dists, nn_idxs = nbr.kneighbors(subF)
        sims = 1 - dists

        for ii, src in enumerate(top_idx):
            close = set(nx.single_source_shortest_path_length(
                G, int(src), cutoff=hop_thresh
            ).keys())

            for rank, dst_j in enumerate(nn_idxs[ii,1:], start=1):
                if sims[ii, rank] < sim_thresh:
                    break
                dst = top_idx[dst_j]
                u, v = sorted((int(src), int(dst)))
                if road_ids[src] == road_ids[dst]:
                    continue
                dxy = np.hypot(*(coords[src] - coords[dst]))
                if dxy < min_dist or dxy > max_dist:
                    continue
                if dst in close:
                    continue
                candidates.add((u, v))

    # trim to budgets
    final, counts = [], Counter()
    for u, v in random.sample(list(candidates), len(candidates)):
        if counts[u] < per_node_cap and counts[v] < per_node_cap:
            final.append((u, v))
            counts[u] += 1
            counts[v] += 1
        if len(final) >= max_edges:
            break

    G2 = G.copy()
    G2.add_edges_from(final, feature_sim=suffix)
    return G2, final

# ────────────────────────────────────────────────────────────
# 4) build_data that masks missing NO2 (no imputation)
# ────────────────────────────────────────────────────────────
def build_data_mask_missing(G, gdf, feature_cols, target_col):
    gdf2 = gdf.reset_index(drop=True)
    G2   = nx.relabel_nodes(G, {old:new for new,old in enumerate(gdf.index)})

    X = StandardScaler().fit_transform(gdf2[feature_cols].values)
    y = gdf2[target_col].values.reshape(-1,1)

    edges = np.array(list(G2.edges())).T
    edge_index = torch.tensor(
        np.concatenate([edges, edges[::-1]], axis=1),
        dtype=torch.long
    )

    data = Data(
        x=torch.tensor(X, dtype=torch.float),
        edge_index=edge_index,
        y=torch.tensor(y, dtype=torch.float)
    )

    valid_idx = np.where(~np.isnan(y.flatten()))[0]
    perm = torch.randperm(len(valid_idx))
    n_train = int(0.8 * len(valid_idx))
    train_idx = valid_idx[perm[:n_train].numpy()]
    test_idx  = valid_idx[perm[n_train:].numpy()]

    train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
    test_mask  = torch.zeros(data.num_nodes, dtype=torch.bool)
    train_mask[torch.tensor(train_idx, dtype=torch.long)] = True
    test_mask [torch.tensor(test_idx,  dtype=torch.long)] = True

    data.train_mask = train_mask
    data.test_mask  = test_mask
    return data

# ────────────────────────────────────────────────────────────
# 5) Define GCN, GAT & train/eval
# ────────────────────────────────────────────────────────────
class GCN(torch.nn.Module):
    def __init__(self, in_c, h_c, out_c):
        super().__init__()
        self.c1 = GCNConv(in_c,  h_c)
        self.c2 = GCNConv(h_c,    h_c)
        self.c3 = GCNConv(h_c,  out_c)
    def forward(self, x, e):
        x = F.relu(self.c1(x,e))
        x = F.relu(self.c2(x,e))
        return F.relu(self.c3(x,e))

class GAT(torch.nn.Module):
    def __init__(self, in_c, h_c, out_c, heads=2):
        super().__init__()
        self.g1 = GATConv(in_c,   h_c,  heads=heads)
        self.g2 = GATConv(h_c*heads, h_c, heads=heads)
        self.g3 = GATConv(h_c*heads, out_c, heads=1, concat=False)
    def forward(self, x, e):
        x = F.elu(self.g1(x,e))
        x = F.elu(self.g2(x,e))
        return self.g3(x,e)

def train_and_eval(model_cls, data, **model_kwargs):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model  = model_cls(**model_kwargs).to(device)
    data   = data.to(device)
    opt    = torch.optim.Adam(model.parameters(), lr=0.01)
    for _ in range(200):
        model.train(); opt.zero_grad()
        out = model(data.x, data.edge_index)
        F.mse_loss(out[data.train_mask], data.y[data.train_mask]).backward()
        opt.step()
    model.eval()
    with torch.no_grad():
        pred = model(data.x, data.edge_index)[data.test_mask].cpu().numpy().flatten()
        true = data.y[data.test_mask].cpu().numpy().flatten()
    mse = mean_squared_error(true, pred)
    return {
        'mse': mse,
        'rmse': np.sqrt(mse),
        'mae': mean_absolute_error(true, pred),
        'r2': r2_score(true, pred),
        'pearson': pearsonr(true, pred)[0]
    }

# ────────────────────────────────────────────────────────────
# 6) “Smart” grid‐search (<..trials) + save outputs
# ────────────────────────────────────────────────────────────
param_grid = {
    'neighbors':    [500, 1000],
    'sim_thresh':   [0.995],
    'min_dist':     [200.0],
    'max_dist':     [1000.0, 2000, 4000, 8000],
    'hop_thresh':   [4],
    'max_edges':    [1000, 2000],
    'per_node_cap': [2],
}
fixed = {'top_n':2000, 'suffix':'smart'}

os.makedirs('outputs', exist_ok=True)
with open('outputs/grid_params.json','w') as f:
    json.dump(param_grid, f, indent=2)

results=[]
for params in tqdm(list(ParameterGrid(param_grid)), desc="Grid Trials"):
    G_aug, _ = augment_grouped_far_knn(
        G, gdf, groups, coords,
        feature_matrix, feature_cols,
        **fixed, **params
    )
    data_aug = build_data_mask_missing(G_aug, gdf, feature_cols, 'NO2d')
    gcn_m = train_and_eval(GCN, data_aug,
                           in_c=len(feature_cols),
                           h_c=16,
                           out_c=1)
    gat_m = train_and_eval(GAT, data_aug,
                           in_c=len(feature_cols),
                           h_c=16,
                           out_c=1,
                           heads=2)
    results.append({**params,
                    **{f'GCN_{k}':v for k,v in gcn_m.items()},
                    **{f'GAT_{k}':v for k,v in gat_m.items()}})

df = pd.DataFrame(results)
df.to_csv('/content/drive/MyDrive/Universiteit Utrecht/Thesis/outputs/grid_search_results.csv', index=False)
print("Grid search complete.")

best = df.loc[df['GCN_rmse'].idxmin()]
raw = best[list(param_grid.keys())].to_dict()
int_keys = {'neighbors','hop_thresh','max_edges','per_node_cap','top_n'}
best_params = {k:int(v) if k in int_keys else float(v) for k,v in raw.items()}
with open('outputs/best_params.json','w') as f:
    json.dump(best_params, f, indent=2)
print("Best params:", best_params)

# ────────────────────────────────────────────────────────────
# 7) Final augmentation, train+predict, save
# ────────────────────────────────────────────────────────────
G_final, _ = augment_grouped_far_knn(
    G, gdf, groups, coords,
    feature_matrix, feature_cols,
    top_n=2000, suffix='final', **best_params
)
data_final = build_data_mask_missing(G_final, gdf, feature_cols, 'NO2d')

def full_predict(model_cls, **kw):
    dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    m = model_cls(**kw).to(dev)
    d = data_final.to(dev)
    opt = torch.optim.Adam(m.parameters(), lr=0.01)
    for _ in range(200):
        m.train(); opt.zero_grad()
        F.mse_loss(m(d.x,d.edge_index)[d.train_mask],
                   d.y[d.train_mask]).backward()
        opt.step()
    m.eval()
    with torch.no_grad():
        return m(d.x,d.edge_index).cpu().numpy().flatten()

gdf['NO2_pred_GCN'] = full_predict(GCN, in_c=len(feature_cols), h_c=16, out_c=1)
gdf['NO2_pred_GAT'] = full_predict(GAT, in_c=len(feature_cols), h_c=16, out_c=1, heads=2)
gdf.to_file('/content/drive/MyDrive/Universiteit Utrecht/Thesis/outputs/road_segments_with_predictions.geojson', driver='GeoJSON')
with open('/content/drive/MyDrive/Universiteit Utrecht/Thesis/outputs/final_params.json','w') as f:
    json.dump(best_params, f, indent=2)

# ────────────────────────────────────────────────────────────
# 8) External Palmes validation & save
# ────────────────────────────────────────────────────────────
palmes_fp = '/content/drive/MyDrive/Universiteit Utrecht/Thesis/data/road_palmes_25m.geojson'
palmes_gdf = gpd.read_file(palmes_fp).to_crs(gdf.crs)
sidx = gdf.sindex
radius = 50
records = []
for _, row in palmes_gdf.iterrows():
    pt = row.geometry
    cands = list(sidx.intersection(pt.buffer(radius).bounds))
    if cands:
        dists = gdf.loc[cands,'geometry'].distance(pt).values
        near  = [cands[i] for i, d in enumerate(dists) if d <= radius]
    else:
        near = []
    if not near:
        near = [gdf.geometry.distance(pt).idxmin()]
    records.append({
        'meas_NO2': row['mean_annual_palmes_no2'],
        'pred_GCN': gdf.loc[near,'NO2_pred_GCN'].mean(),
        'pred_GAT': gdf.loc[near,'NO2_pred_GAT'].mean(),
        **row.drop('geometry').to_dict()
    })

palmes_val = gpd.GeoDataFrame(records, geometry=palmes_gdf.geometry, crs=gdf.crs)
palmes_val['err_GCN'] = palmes_val['pred_GCN'] - palmes_val['meas_NO2']
palmes_val['err_GAT'] = palmes_val['pred_GAT'] - palmes_val['meas_NO2']
palmes_val.to_file('/content/drive/MyDrive/Universiteit Utrecht/Thesis/outputs/palmes_external_validation.geojson', driver='GeoJSON')
with open('/content/drive/MyDrive/Universiteit Utrecht/Thesis/outputs/external_params.json','w') as f:
    json.dump(best_params, f, indent=2)

for mod in ['GCN','GAT']:
    mse = mean_squared_error(palmes_val['meas_NO2'], palmes_val[f'pred_{mod}'])
    print(f"{mod} Palmes RMSE:", np.sqrt(mse))

Grid Trials:   0%|          | 0/16 [00:00<?, ?it/s]

KeyboardInterrupt: 